In [ ]:
%pip install --upgrade pandas
%pip install --upgrade numpy
%pip install --upgrade matplotlib
%pip install --upgrade seaborn
%pip install --upgrade scikit-learn-intelex
%pip install --upgrade dpcpp-cpp-rt
%pip install --upgrade xgboost
%pip install --upgrade sklearn-ex

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

Importing Libraries